In [ ]:
%pip install openai "androguard>=4" Pillow

## V3


第三个版本主要更新的内容为：

    1. 将生成的level3 task维持在15步左右
    2. 尝试调整加入功能点


In [1]:
from androguard.util import set_log

set_log("ERROR")  # 关闭琐碎的DEBUG输出
from androguard.core.apk import APK
import os
import re
import openai

openai.api_key = "sk-MBHdeyudZupuva1U90Cb85B7D65849A48c854a2f2f7a00A9"
openai.base_url = "https://api.pumpkinaigc.online/v1/"

model = "gpt-4"
# model = "claude_3_opus"

openai.default_headers = {"x-foo": "true"}


# https://androguard.readthedocs.io/en/latest/intro/gettingstarted.html

In [34]:
prompt_template = """当前app的名字是：{}，包名是：{}
它有如下的activity，activity指的是是一个应用程序组件，提供一个屏幕，用户可以用来交互为了完成某项任务：
{}

请你根据app的名字和activity，列举出该app的能够在几步的操作内完成的task，举个例子，如果app的名字是视频软件的话，会有如下的task：
1. 观看视频
2. 搜索视频
3. 对搜索结果进行筛选、排序
4. 评论视频
5. 点赞视频
6. 收藏视频

请注意，你只能列举task，而且需要中文输出，不能输出任何多余的信息，并且要尽可能多地列举task。你列举的task应该是用户日常会使用到的功能。
"""


prompt_template2 = """
以下是{}可供参考的基础操作，请基于此，构建尽可能多的{}的级别3的操作，注意保证用户执行这个任务序列的长度大约是5步左右。
{}


完整的级别3样例为：
• 淘宝
- 级别 1: 基础操作 - 搜索一台华为 Mate60。
- 级别 2: 复杂操作 - 添加一台华为 Mate60 至购物车，并确认购物车内有此商品。
- 级别 3: 多项任务组合 - 添加一台 1TB 的华为 Mate60 和它对应的手机壳至购物车，并确认购物车内有这些商品。

• 高德地图
- 级别 1: 基础操作 - 搜索一家附近的麦当劳。
- 级别 2: 复杂操作 - 查找到一家附近麦当劳的步行路线。
- 级别 3: 多项任务组合 - 查找到一家附近麦当劳的步行路线，途径一家附近的肯德基。


• 淘宝
- 级别 1: 基础操作 - 搜索一顶网球帽。 
- 级别 2: 复杂操作 - 搜索一顶网球帽，添加到购物车，并确定购物车中存在这个商品。
- 级别 3: 多项任务组合 - 搜索一顶具有多种配色的网球帽，任意选择两种配色，添加到购物车，并确定购物车中存在这个商品。

请注意，你只能列举操作，不能输出任何多余的信息，并且要尽可能多地列举操作。你列举的操作应该是用户日常会使用到的功能，贴近实际的场景，此外注意你定义的任务应该具有唯一性，指定任务中的关键词，使用户执行的时候只有一种可能。
比如应该出现的是类似于这样的例子：
1. 搜索一个名为“超级小桀”的用户，关注该用户，观看其发布的最新视频，并发送一条弹幕。
而不是：
1. 搜索用户，关注用户，发送弹幕

"""  # app name, app name, task_str


# 把级别3的task拆分
prompt_template3 = """注意输出的格式要按照下面的例子：
1. 搜索《鬼灭之刃》番剧，添加到收藏，开始播放第一集，再搜索相关讨论话题并参与讨论。
   - 级别1：搜索《鬼灭之刃》番剧。
   - 级别2：搜索《鬼灭之刃》番剧，添加到收藏。
   - 级别3：搜索《鬼灭之刃》番剧，添加到收藏，开始播放第一集，再搜索相关讨论话题并参与讨论。

2. 搜索一部特定的漫画如《航海王》，开始播放第一章，收藏该漫画，加入到个人漫画收藏夹，并且发一条评论。
   - 级别1：搜索一部特定的漫画如《航海王》。
   - 级别2：搜索一部特定的漫画如《航海王》，开始播放第一章。
   - 级别3：搜索一部特定的漫画如《航海王》，开始播放第一章，收藏该漫画，加入到个人漫画收藏夹，并且发一条评论。

你需要根据级别1、级别2和级别3任务之间的递进关系，对于下面展示的一组级别3的任务，针对其中每个级别3的任务构造对应级别1和级别2以及级别3的任务：
{}

"""  # task_str

In [35]:
def chat(prompt):
    completion = openai.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
        temperature=1,
    )
    return completion.choices[0].message.content


def apk_to_simple_task(apk_object):
    def parse_task(text):
        pattern = r"\d+\.+(.*)"
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            matches[i] = match.strip()
        return matches

    a = apk_object
    activity = [act for act in a.get_activities() if "sdk" not in act]
    activity_str = "\n".join(activity)
    prompt = prompt_template.format(a.get_app_name(), a.get_package(), activity_str)
    task = parse_task(chat(prompt))
    return task


def simple_task_to_level3_task(simple_task_list, a):
    def parse_task(text):
        pattern = r"\d+\.+(.*)"
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            matches[i] = match.strip()
        return matches

    task_str = ""
    for i, task in enumerate(simple_task_list):
        task_str += f"{i+1}. {task}\n"
    p = prompt_template2.format(a.get_app_name(), a.get_app_name(), task_str)
    c = chat(p)
    t = parse_task(c)
    return t


def level3_to_level123_task(level3_task_list, a):
    def parse_task(text):
        pattern = r"\d+\.+(.*)"
        matches = re.findall(pattern, text)
        for i, match in enumerate(matches):
            matches[i] = match.strip()
        return matches

    task_str = ""
    for i, task in enumerate(level3_task_list):
        task_str += f"{i+1}. {task}\n"
    p = prompt_template3.format(task_str)
    c = chat(p)
    # t = parse_task(c)
    return c


def save_to_md(a, level123_task_list, level3_task_list, tasklist):
    fn = f"{a.get_package()}.md"
    with open(fn, "w") as f:
        f.write(f"# {a.get_app_name()}\n\n")
        f.write(f"包名：{a.get_package()}\n\n")
        f.write(f"main activity: {a.get_main_activity()}\n\n")

        f.write("## 生成得到的任务\n\n")
        f.write(f"{level123_task_list}\n")

        f.write("## 功能点\n\n")
        for i, task in enumerate(tasklist):
            f.write(f"{i+1}. {task}\n\n")

        # f.write("## 级别3的任务\n\n")
        # for i, task in enumerate(level3_task_list):
        #     f.write(f"{i+1}. {task}\n\n")


def process_apk(apk_path):
    a = APK(apk_path)
    tasklist = apk_to_simple_task(a)
    level3_task_list = simple_task_to_level3_task(tasklist, a)
    print("level 3 task generated")
    level123_task_list = level3_to_level123_task(level3_task_list, a)
    save_to_md(a, level123_task_list, level3_task_list, tasklist)

In [36]:
model = "gpt-4o"

In [37]:
folder = rf"/root/data/huawei/apk"
fn = "QQMusic.apk"

apk_fp = os.path.join(folder, fn)  # apk文件路径

process_apk(apk_fp)  # 会将task列表保存到当前目录下的md文件中（文件名就是apk的包名）

level 3 task generated


In [ ]:
%pip install -q pyperclip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyperclip

s = """好评优先
餐厅
"""

result = '"' + str(s.strip().split("\n")) + '"'
pyperclip.copy(result)
print(result, "已经复制到剪贴板")

"['好评优先', '餐厅']" 已经复制到剪贴板


In [12]:
import openai
import base64

client = openai.OpenAI(
    api_key="sk-MBHdeyudZupuva1U90Cb85B7D65849A48c854a2f2f7a00A9",
    base_url="https://api.pumpkinaigc.online/v1",
    timeout=2000,
    max_retries=3,
)
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """
对于淘宝、高德地图、美团、QQ 音乐、哔哩哔哩，这五个应用，设计参考任务/功能项，下面给出了一些实例
∗ 淘宝
· 搜索一个水杯，价格在 100 元以内，将它加入购物车
· 搜索一条白色裙子，价格在 400 元以内，s 码或均码，将它
加入购物车
· 加 3 瓶农夫山泉维他命水到购物车
∗ 高德地图
· 导航去鸟巢
· 从鸟巢回来的路上我想去洗车
∗ 美团
· 打开美团外卖，点一杯星巴克并加入购物车
· 在美团上看看最近热映的电影
· 查一下沙丘 2 这部电影还有票吗
· 找一个美团单车附近的停车点
· 找一家附近的理发店，并预约明天 11 点的烫发
∗ QQ 音乐
· 搜索并播放五月天的干杯
· 播放一首干杯
· 播放一首五月天的歌
∗ 哔哩哔哩
· 关注食贫道，进入他的主页，给播放量最高的视频一键三连
· 我要看人世间第三集，1.5 倍速播放
· 查看我的历史记录并继续播放视频
· 删除搜索记录
· 放一个最新的番剧

你的任务是对于每个APP给出10个功能点，类似于下面：
淘宝

促销活动提醒： 根据用户的购买历史和偏好，提供个性化的促销活动提醒，包括优惠券、限时折扣等。
商品比价功能： 提供类似商品的价格比较功能，帮助用户选择性价比更高的商品。
拼团购买： 支持用户参与或发起拼团活动，享受团购优惠。
二手市场交易： 提供二手商品交易平台，用户可以发布二手商品信息并进行交易。
商品追踪与价格监控： 允许用户追踪关注的商品，并设置价格变动提醒，帮助用户把握购物时机。
高德地图

实时路况播报： 在导航过程中提供实时路况信息，帮助用户选择最佳行驶路线。
公共交通路线查询： 提供公交、地铁等公共交通路线查询和导航功能。
停车位预订： 允许用户查询附近停车位信息，并预订停车位，避免停车难题。
景点导览： 提供景点介绍、导览路线等信息，帮助用户规划游览行程。
实时天气信息： 提供目的地及周边地区的实时天气信息，帮助用户合理安排行程。
美团

外卖配送追踪： 在下单后提供外卖配送的实时追踪功能，让用户随时了解订单状态。
美食推荐： 根据用户口味和偏好，推荐附近美食店铺和特色菜品。
优惠活动查看： 提供美团平台上各类商家的优惠活动信息，包括折扣、满减等。
美团会员权益： 展示美团会员的专属权益和优惠，吸引用户加入会员。
实时订单评价： 提供用户对已完成订单进行评价的功能，帮助其他用户选择商家或商品。
QQ音乐

歌词显示： 在播放音乐时显示歌曲的歌词，让用户更好地理解和跟唱歌曲。
歌手资料查看： 提供歌手的详细资料、专辑列表等信息，让用户更好地了解自己喜爱的歌手。
定时停止播放： 提供定时停止播放功能，让用户在睡前设定时间后音乐自动停止播放。
音乐分享： 支持用户将正在听的音乐分享到社交平台，与好友分享音乐心情。
歌曲下载： 允许用户购买或下载喜欢的歌曲，以便在无网络环境下进行收听。
哔哩哔哩

视频创作工具： 提供丰富的视频编辑工具，支持用户进行视频剪辑、特效添加等操作。
直播功能： 允许用户观看和参与直播节目，与主播互动交流。
虚拟礼物赠送： 用户可以通过购买虚拟礼物为喜欢的主播送礼，支持主播。
视频字幕翻译： 提供视频字幕翻译功能，支持用户浏览和理解外语视频内容。
个性化推荐视频创作者： 根据用户的观看历史和兴趣，推荐适合用户口味的视频创作者和内容。

""",
                },
            ],
        },
    ],
    model="gpt-4o",
    temperature=1,
    max_tokens=2000,
)

In [13]:
print(response.choices[0].message.content)

### 淘宝
1. **促销活动提醒**： 根据用户的购买历史和偏好，提供个性化的促销活动提醒，包括优惠券、限时折扣等。
2. **商品比价功能**： 提供类似商品的价格比较功能，帮助用户选择性价比更高的商品。
3. **拼团购买**： 支持用户参与或发起拼团活动，享受团购优惠。
4. **二手市场交易**： 提供二手商品交易平台，用户可以发布二手商品信息并进行交易。
5. **商品追踪与价格监控**： 允许用户追踪关注的商品，并设置价格变动提醒，帮助用户把握购物时机。
6. **商品收藏**： 用户可以收藏喜欢的商品，方便以后查看和购买。
7. **商品评价**： 用户可以评价已购买的商品，帮助其他用户做出选择。
8. **订单追踪**： 实时追踪订单状态，包括发货、运输和签收信息。
9. **客户服务聊天**： 提供在线客服聊天功能，解决用户在购物过程中遇到的问题。
10. **虚拟试衣间**： 提供虚拟试衣功能，通过AR技术帮助用户试穿衣服，提高购物体验。

### 高德地图
1. **实时路况播报**： 在导航过程中提供实时路况信息，帮助用户选择最佳行驶路线。
2. **公共交通路线查询**： 提供公交、地铁等公共交通路线查询和导航功能。
3. **停车位预订**： 允许用户查询附近停车位信息，并预订停车位，避免停车难题。
4. **景点导览**： 提供景点介绍、导览路线等信息，帮助用户规划游览行程。
5. **实时天气信息**： 提供目的地及周边地区的实时天气信息，帮助用户合理安排行程。
6. **街景查看**： 提供街景查看功能，帮助用户了解目的地的实际情况。
7. **步行&骑行导航**： 除了驾车导航，还提供步行和骑行的导航选项。
8. **救援服务**： 提供紧急情况下的救援服务，例如道路救援和紧急电话。
9. **离线地图下载**： 用户可以在有网络时下载地图，在无网络条件下使用。
10. **目的地分享**： 允许用户将目的地或路线分享给好友，方便共同出行。

### 美团
1. **外卖配送追踪**： 在下单后提供外卖配送的实时追踪功能，让用户随时了解订单状态。
2. **美食推荐**： 根据用户口味和偏好，推荐附近美食店铺和特色菜品。
3. **优惠活动查看**： 提供美团平台上各类商家的优惠活动信息，包括折扣、满减等。
4. **美团会员